<a href="https://colab.research.google.com/github/giyeongyoon/3rd_AGC/blob/master/two_stage_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고문헌

*   Estimation of Greenhouse Lettuce Growth Indices Based on a
Two-Stage CNN Using RGB-D Images
[Data Availability](https://data.4tu.nl/articles/dataset/3rd_Autonomous_Greenhouse_Challenge_Online_Challenge_Lettuce_Images/15023088/1)

In [ ]:
%%capture
!pip install albumentations==1.1.0
!pip install agml

Import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os
import cv2
import albumentations as A
import pandas as pd
import matplotlib.pyplot as plt

Download 2021 Autonomous Greenhouse Challenge dataset

In [ ]:
import agml
loader = agml.data.AgMLDataLoader('autonomous_greenhouse_regression', dataset_path = './')

Define data and output directories

In [ ]:
sav_dir='model_weights/'
if not os.path.exists(sav_dir):
    os.mkdir(sav_dir)
# Comment these two lines and uncomment the next two if you've already croppped the images to another directory
RGB_Data_Dir   = './autonomous_greenhouse_regression/images/'
Depth_Data_Dir = './autonomous_greenhouse_regression/depth_images/'


# RGB_Data_Dir='./autonomous_greenhouse_regression/cropped_images/'
# Depth_Data_Dir='./autonomous_greenhouse_regression/cropped_depth_images/'


JSON_Files_Dir = './autonomous_greenhouse_regression/annotations.json'

Crop

In [ ]:
# import matplotlib.pyplot as plt
min_x=650
max_x=1450
min_y=200
max_y=900
cropped_img_dir='./autonomous_greenhouse_regression/cropped_images/'

cropped_depth_img_dir='./autonomous_greenhouse_regression/cropped_depth_images/'

if not os.path.exists(cropped_img_dir):
    os.mkdir(cropped_img_dir)

if not os.path.exists(cropped_depth_img_dir):
    os.mkdir(cropped_depth_img_dir)

for im in os.listdir(RGB_Data_Dir):
    img = cv2.imread(RGB_Data_Dir+im)
    crop_img = img[min_y:max_y,min_x:max_x]
    cv2.imwrite(cropped_img_dir+im, crop_img)

for depth_im in os.listdir(Depth_Data_Dir):
    depth_img = cv2.imread(Depth_Data_Dir+depth_im, 0)
    crop_depth_img = depth_img[min_y:max_y,min_x:max_x]
    cv2.imwrite(cropped_depth_img_dir+depth_im, crop_depth_img)

RGB_Data_Dir   = cropped_img_dir
Depth_Data_Dir = cropped_depth_img_dir

Check the targets

In [ ]:
# df= pd.read_json(JSON_Files_Dir)
# row = df.iloc[0]
# print(list(row['outputs']['regression']))
# print(list(row['outputs']['regression'].values()))

Create PyTorch dataset, create PyTorch dataloader, and split train/val/test

In [ ]:
split_seed = 12
num_epochs = 400

In [ ]:
class GreenhouseDataset(Dataset):
    def __init__(self, rgb_dir, d_dir, jsonfile_dir, rgb_transforms=None, d_transforms=None):

        self.df= pd.read_json(jsonfile_dir)
        # flatten_json is a custom function to flat the nested json files!

        self.rgb_transforms = rgb_transforms
        self.d_transforms = d_transforms
        self.rgb_dir = rgb_dir
        self.d_dir = d_dir
        self.num_outputs = len(self.df.iloc[0]['outputs']['regression'])


    def __getitem__(self, idx):
        # load images
        row=self.df.iloc[idx]

        rgb = plt.imread(self.rgb_dir+row['image'])
        depth = plt.imread(self.d_dir+row['depth_image'])
        depth = np.expand_dims(depth, 2)

        target = list(row['outputs']['regression'].values())

        #make sure your img and mask array are in this format before passing into albumentations transforms, img.shape=[H, W, C]
        if self.rgb_transforms is not None:
            aug_rgb = self.rgb_transforms(image=rgb)
            rgb = aug_rgb['image']
        elif self.d_transforms is not None:
            aug_depth = self.d_transforms(image=depth)
            depth = aug_depth['image']

        rgb = np.transpose(rgb, (2,0,1))
        depth = np.transpose(depth, (2,0,1))

        #pytorch wants a different format for the image ([C, H, W])
        rgb = torch.as_tensor(rgb, dtype=torch.float32)
        depth = torch.as_tensor(depth, dtype=torch.float32)
        target=torch.as_tensor(target, dtype=torch.float32)

        return rgb, depth, target

    def __len__(self):
        return len(self.df)

In [ ]:
## FIGURE OUT HOW TO CROP ALL THE IMAGES TO GET RID OF EXTRANIOUS PIXELS
def get_transforms(train, means, stds):
    if train:
        transforms = A.Compose([
        # A.Crop(x_min=650, y_min=200, x_max=1450, y_max=900, always_apply=False, p=1.0),
        A.Flip(p=0.5),
        A.ShiftScaleRotate(always_apply=False, p=0.5, shift_limit=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-5, 5), interpolation=0, border_mode=0, value=means, mask_value=None),
        A.Normalize(mean=means, std=stds, max_pixel_value=1.0, always_apply=False, p=1.0)
        ])
    else:
        transforms =  A.Compose([
        # A.Crop(x_min=650, y_min=200, x_max=1450, y_max=900, always_apply=False, p=1.0),
        A.Normalize(mean=means, std=stds, max_pixel_value=1.0, always_apply=False, p=1.0)
        ])
    return transforms

In [ ]:
# Instantiate the PyTorch datalaoder the autonomous greenhouse dataset.
dataset = GreenhouseDataset(rgb_dir = RGB_Data_Dir,
                            d_dir = Depth_Data_Dir,
                            jsonfile_dir = JSON_Files_Dir,
                            rgb_transforms = get_transforms(train=False, means=[0,0,0],stds=[1,1,1]),
                            d_transforms = get_transforms(train=False, means=[0,0,0],stds=[1,1,1]))

# Remove last 50 images from training/validation set. These are the test set.
dataset.df= dataset.df.iloc[:-50]

# Split train and validation set. Stratify based on variety.
train_split, val_split = train_test_split(dataset.df,
                                          test_size = 0.2,
                                          random_state = split_seed,
                                          stratify = dataset.df['outputs'].str['classification']) #change to None if you don't have class info
train = torch.utils.data.Subset(dataset, train_split.index.tolist())
val   = torch.utils.data.Subset(dataset, val_split.index.tolist())

# Create train and validation dataloaders
train_loader = torch.utils.data.DataLoader(train, batch_size=6, num_workers=6, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val,   batch_size=6, shuffle=False, num_workers=6)


Determine the mean and standard deviation of images for normalization (Only need to do once for a new dataset)

In [ ]:
# this part is just to check the MEAN and STD of the dataset (dont run unless you need mu and sigma)

n_rgb = 0
n_depth = 0
mean_rgb = 0.
std_rgb = 0.
mean_depth = 0.
std_depth = 0.
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=False, num_workers=12)
for rgb, depth, _ in dataloader:

    # Rearrange batch to be the shape of [B, C, W * H]
    rgb = rgb.view(rgb.size(0), rgb.size(1), -1)
    depth = depth.view(depth.size(0), depth.size(1), -1)
    # Update total number of images
    n_rgb += rgb.size(0)
    n_depth += depth.size(0)
    # Compute mean and std here
    mean_rgb += rgb.mean(2).sum(0)
    std_rgb += rgb.std(2).sum(0)
    mean_depth += depth.mean(2).sum(0)
    std_depth += depth.std(2).sum(0)

# Final step
mean_rgb /= n_rgb
std_rgb /= n_rgb
mean_depth /= n_depth
std_depth /= n_depth

print('Mean of RGB: '+ str(mean_rgb))
print('Standard Deviation of RGB', str(std_rgb))
print('Mean of Depth: '+ str(mean_depth))
print('Standard Deviation of Depth', str(std_depth))

Copy the output of the previous cells into here to avoid needing to redetermine mean and std every time

In [ ]:
dataset.means = [0.5482, 0.4620, 0.3602, 0.0127]  #these values were copied from the previous cell
dataset.stds = [0.1639, 0.1761, 0.2659, 0.0035]   #copy and paste the values to avoid having
                                                  # to rerun the previous cell for every iteration

Set device

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

Model

In [ ]:
# !pip install torchsummary
# import torchsummary

# model = models.resnet50(pretrained=True)
# model = model.cuda()
# torchsummary.summary(model, (3, 224, 224))

In [ ]:
class FirstStageModel(nn.Module):
    def __init__(self):
        super(FirstStageModel, self).__init__()
        # RGB Model
        self.rgb_processing_block = nn.Sequential(nn.Conv2D(3, 32, kernel_size=3, stride=2, padding=1),
                                       nn.Conv2D(32, 3, kernel_size=1),
                                       nn.AdaptiveAvgPool2d((224, 224)))
        self.rgb_encoder = models.resnet50(pretrained=True)
        self.rgb_regressor = nn.Sequential(nn.ReLU(replace=True),
                                           nn.Dropout(0.5),
                                           nn.Linear(1000, 256),
                                           nn.ReLU(replace=True),
                                           nn.Dropout(0.5),
                                           nn.Linear(256, 4))


        # Depth Model
        self.depth_processing_block = nn.Sequential(nn.Conv2D(1, 32, kernel_size=3, stride=2, padding=1),
                                         nn.Conv2D(32, 1, kernel_size=1),
                                         nn.AdaptiveAvgPool2d((224, 224)))
        self.depth_encoder = models.resnet50(pretrained=False)
        self.depth_regressor = nn.Sequential(nn.ReLU(replace=True),
                                             nn.Dropout(0.5),
                                             nn.Linear(1000, 256),
                                             nn.ReLU(replace=True),
                                             nn.Dropout(0.5),
                                             nn.Linear(256, 1))

        self.final = nn.Sequential(nn.Dropout(0.5),
                                   nn.Linear(5, 2048),
                                   nn.ReLU(replace=True)
                                   nn.Dropout(0.5),
                                   nn.Linear(2048, 3),
                                   nn.Dropout(0.5))

    def forward(self, rgb, depth):
        rgb_out = self.rgb_processing_block(rgb)
        rgb_out = self.rgb_encoder(rgb_out)
        rgb_out = self.rgb_regressor(rgb_out)

        depth_out = self.depth_processing_block(depth)
        depth_out = self.depth_encoder(depth_out)
        output2 = self.depth_regressor(depth_out)  # height

        output1 = torch.cat([rgb_out, depth_out], dim=1)
        output1 = self.final(output1)  # fresh weight, dry weight, diameter

        return output1, output2

In [ ]:
class SecondStageModel(nn.Module):
    def __init__(self):
        super(SecondStageModel, self).__init__()
        self.regressor1 = nn.Sequential(nn.Linear(4, 2048),
                                        nn.ReLU(replace=True),
                                        nn.Dropout(0.5),
                                        nn.Linear(2048, 2048),
                                        nn.ReLU(replace=True),
                                        nn.Dropout(0.5),
                                        nn.Linear(2048, 1))  # dry weight
        self.regressor2 = nn.Sequential(nn.Linear(4, 2048),
                                        nn.ReLU(replace=True),
                                        nn.Dropout(0.5),
                                        nn.Linear(2048, 2048),
                                        nn.ReLU(replace=True),
                                        nn.Dropout(0.5),
                                        nn.Linear(2048, 1))  # leaf area

    def forward(self, output1, output2):
        x = torch.cat([output1, output2], dim=1)
        output3 = self.regressor1(x)
        output4 = self.regressor2(x)
        return output3, output4

In [ ]:
first_stage_model = FirstStageModel()
second_stage_model = SecondStageModel()

Hyperparameter

In [ ]:
lr = 0.001
epochs = 200
batch_size = 32

Loss and optimizer

In [ ]:
criterion_stage1 = nn.MSELoss()
criterion_stage2 = nn.MSELoss()

optimizer_stage1 = optim.Adam(first_stage_model.parameters(), lr=lr)
optimizer_stage2 = optim.Adam(second_stage_model.parameters(), lr=lr)

Train

In [ ]:
def validate(first_stage_model, second_stage_model, dataset, device, sav_dir,
             criterion_stage1, criterion_stage2, writer, epoch, val_loader, best_val_loss):
    current_val_loss = 0
    # training_val_loss=0s

    first_stage_model.eval()
    second_stage_model.eval()
    print('Validating and Checkpointing!')

    dataset.rgb_transforms = get_transforms(train=True, means=dataset.means[:3], stds=dataset.stds[:3])
    dataset.d_transforms = get_transforms(train=True, means=dataset.means[3:], stds=dataset.stds[3:])

    with torch.no_grad():
        for i, (rgb, depth, label) in enumerate(val_loader):
            rgb = rgb.to(device)
            depth = depth.to(device)
            label = label.to(device)

            pred1, pred2 = first_stage_model(rgb, depth)
            combined_output = torch.cat([pred1, pred2], dim=1)
            pred3, pred4 = second_stage_model(combined_output)

            loss_stage1 = criterion_stage1(pred1, label[:, [0,2]]) + criterion_stage1(pred2, label[:, 3])
            loss_stage2 = criterion_stage2(pred3, label[:, 1]) + criterion_stage2(pred4, label[:, 4])
            total_loss = loss_stage1 + loss_stage2
            # acc=nmse(preds.detach(), targets)
            current_val_loss = current_val_loss + total_loss.item()
            # training_val_loss=training_val_loss+loss.detach().cpu().numpy()

        # writer.add_scalar("MSE Loss/val", training_val_loss, epoch)
        writer.add_scalar("NMSE Loss/val", current_val_loss, epoch)

    if current_val_loss < best_val_loss or epoch == 0:
        best_val_loss = current_val_loss
        torch.save(model.state_dict(), sav_dir+'bestmodel' + '.pth') # should be fixed!
        print('Best model Saved! Val NMSE: ', str(best_val_loss))
        with open('run.txt', 'a') as f:
            f.write('\n')
            f.write('Best model Saved! Val NMSE: '+ str(best_val_loss))

    else:
        print('Model is not good (might be overfitting)! Current val NMSE: ', str(current_val_loss), 'Best Val NMSE: ', str(best_val_loss))
        with open('run.txt', 'a') as f:
            f.write('\n')
            f.write('Model is not good (might be overfitting)! Current val NMSE: '+ str(current_val_loss)+ 'Best Val NMSE: '+ str(best_val_loss))
    return best_val_loss

In [ ]:
for epoch in range(epochs):
    first_stage_model.train()
    second_stage_model.train()

    first_stage_model.to(device)
    second_stage_model.to(device)

    dataset.rgb_transforms = get_transforms(train=True, means=dataset.means[:3], stds=dataset.stds[:3])
    dataset.d_transforms = get_transforms(train=True, means=dataset.means[3:], stds=dataset.stds[3:])

    best_val_loss = 9999999 # initial dummy value
    current_val_loss = 0

    writer = SummaryWriter()
    start = time.time()

    for i, (rgb, depth, label) in enumerate(train_loader):
        print('Epoch: ', str(epoch + 1), ', Time Elapsed: ', str((time.time()-start)/60), ' mins')

        rgb = rgb.to(device)
        depth = depth.to(device)
        label = label.to(device)  # ['FreshWeightShoot', 'DryWeightShoot', 'Height', 'Diameter', 'LeafArea']

        # Forward pass - First stage
        pred1, pred2 = first_stage_model(rgb, depth)  # pred1: fresh weight, dry weight, diameter
                                                          # pred2: height

        # Combine outputs for second stage
        combined_output = torch.cat([pred1, pred2], dim=1)

        # Forward pass - Second stage
        pred3, pred4 = second_stage_model(combined_output)  # pred3: dry weight
                                                                # pred4: leaf area

        # Calculate loss
        loss_stage1 = criterion_stage1(pred1, label[:, [0,2]]) + criterion_stage1(pred2, label[:, 3])
        loss_stage2 = criterion_stage2(pred3, label[:, 1]) + criterion_stage2(pred4, label[:, 4])
        total_loss = loss_stage1 + loss_stage2

        # Backward pass and optimization
        optimizer_stage1.zero_grad()
        optimizer_stage2.zero_grad()
        total_loss.backward()
        optimizer_stage1.step()
        optimizer_stage2.step()

        print(f'Epoch {epoch+1}/{epochs}, Batch {i+1}/{len(train_loader)}, Loss1: {loss_stage1.item()}, Loss2: {loss_stage2.item()}, Total_Loss: {total_loss.item()}')

        best_val_loss = validate(first_stage_model, second_stage_model, dataset, device, sav_dir,
                                 criterion_stage1, criterion_stage2, writer, epoch, val_loader, best_val_loss)